# Lab 9: Reconstructing Earth's gravity field
---

## Submission

Please upload your completed notebook (renamed with your name) to Canvas as usual.

## Setup

Please run the code cell below to download the _legendre_schmidt_ function into your working directory.

In [ ]:
# Use curl to obtain legendre_schmidt.py, which contains the function legendre_schmidt()
!curl -O -s https://raw.githubusercontent.com/uafgeoteach/GEOS631_FoG/master/labs/lab_08/legendre_schmidt.py
!pip install pyshtools

## Introduction

In this lab we will use spherical harmonics to compute and visualize Earth's gravity field as well as the geoid height. The gravity model we'll be using for this lab is the Earth Gravitational Model from 1996 (EGM96), which is visualized below:

<a title="F. G. Lemoine, S. C. Kenyon, J. K. Factor, R.G. Trimmer, N. K. Pavlis, D. S. Chinn, C. M. Cox, S. M. Klosko, S. B. Luthcke, M. H. Torrence, Y. M. Wang, R. G. Williamson, E. C. Pavlis, R. H. Rapp and T. R. Olson, NASA Goddard Space Flight Center, Greenbelt, Maryland, 20771 USA, July 1998., Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Earth_Gravitational_Model_1996.png"><img width="512" alt="Earth Gravitational Model 1996" src="https://upload.wikimedia.org/wikipedia/commons/3/3b/Earth_Gravitational_Model_1996.png"></a>

A function to read the EGM96 model is provided as `read_EGM96()`. Before you can compute anomalies, you will need to subtract out the WGS84 reference gravity field. A fragment of code that does this is given below — WGS84 includes terms for J2, J4, J6, J8, and J10. EGM96 is complete to degree and order 360. When you evaluate the gravity field, use the maximum degree and order that your Colab runtime will handle within a reasonable time. Use a smaller maximum degree (around 50 or so) for code development and then increase it for the final result.

In [ ]:
import numpy as np


def read_EGM96(filename='egm96_to360.ascii'):
    """Read the EGM96 gravity field from the file egm96_to360.ascii.
    
    The file is assumed to be in the current directory unless the
    name of the file is passed as an argument.
    
    Args:
        filename (str): Path to egm96_to360.ascii
    
    Returns:
        Tuple containing:
        
        * GM — Product of the Earth's mass and the gravitational constant
        * a — Semi-major axis of WGS84 ellipsoid
        * Cnm — Coefficients for cosine longitude term
        * Snm — Coefficients for sine longitude term
    """

    # Define the constants used in this gravity field
    GM = 3.986004415e14  # [m^3 s^-2]
    a = 6378136.3  # [m]

    # Start by setting Cnm and Snm to zeros (degree and order 360)
    max_degree = 360
    Snm = np.zeros((max_degree + 1, max_degree + 1))
    Cnm = np.zeros((max_degree + 1, max_degree + 1))

    # Open the file and read the coefficients
    coeffs = np.genfromtxt(filename, dtype=None)

    # coeffs now contains all the numerical values in the file
    # Each line is (n, m, Cnm, Snm, sig_Cnm, sig_Snm)
    for line in coeffs:
        n = line[0]
        m = line[1]
        Cnm[n, m] = line[2]
        Snm[n, m] = line[3]

    return GM, a, Cnm, Snm

In [ ]:
"""
Download and read EGM96. We read the nominal values of GM, a (Earth radius), and the
spherical harmonic coefficients Cnm and Snm. Each of these have shape
(361, 361) for EGM96, which is complete to degree and order 360.
"""

!curl -O --ftp-ssl ftp://gdc.cddis.eosdis.nasa.gov/pub/egm96/general_info/egm96_to360.ascii
        
GM, Re, Cnm, Snm = read_EGM96()
g0 = GM / Re**2

# Subtract the even zonals that are part of the ellipsoid definition for WGS84

Cnm[2, 0]  = Cnm[2, 0]  + 0.108262982131e-2  / np.sqrt(5)   # J2
Cnm[4, 0]  = Cnm[4, 0]  - 0.237091120053e-5  / np.sqrt(9)   # J4
Cnm[6, 0]  = Cnm[6, 0]  + 0.608346498882e-8  / np.sqrt(13)  # J6
Cnm[8, 0]  = Cnm[8, 0]  - 0.142681087920e-10 / np.sqrt(17)  # J8
Cnm[10, 0] = Cnm[10, 0] + 0.121439275882e-13 / np.sqrt(21)  # J10

The $C_{nm}$ and $S_{nm}$ coefficients provided by `read_EGM96()` are those that describe the gravitational potential:

$$U = \frac{GM}{r} \sum_{n=0}^\infty \sum_{m=0}^n \left(\frac{R_e}{r}\right)^n [C_{nm}\cos(m\phi) + S_{nm}\sin (m\phi)]P_{nm}(\cos\theta)$$

The equation for gravity anomalies can be derived by solving the **geodetic boundary-value problem**. The gravity anomaly is defined as the observed gravity on the actual surface of the Earth minus the prediction of the reference gravity model at the surface of the geoid. This problem can be linearized and after taking $r = R_e$, it results in the following expression, where $\delta C_{nm}$ and $\delta S_{nm}$ are the $C_{nm}$ and $C_{nm}$ coefficients after subtraction of the reference gravity field:

$$\Delta g = \frac{GM}{R_e^2} \sum_{n=2}^\infty \sum_{m=0}^n (n-1) [\delta C_{nm}\cos(m\phi) + \delta S_{nm}\sin (m\phi)]P_{nm}(\cos\theta)$$

The summation starts at $n=2$ because the $n=0$ term is completely explained by the reference gravity field, and there is no $n=1$ term for the static gravity field as we have chosen the center of mass of the Earth system as the origin.

The equation for the geoid height $N$ is simpler. From Brun's theorem, $N = U/g_0$, which gives (evaluated at $r=R_e$)

$$N = R_e \sum_{n=2}^\infty \sum_{m=0}^n [\delta C_{nm}\cos(m\phi) + \delta S_{nm}\sin (m\phi)]P_{nm}(\cos\theta)\,.$$

## Your task

Make a global map of both the gravity anomalies and geoid height using a reasonable grid for latitude and longitude (about 200 by 400 points or more will do). Then make similar maps for Alaska using a denser computational grid. For all maps, use a suitable map projection and add coastlines using the _cartopy_ package (see https://scitools.org.uk/cartopy/docs/v0.15/matplotlib/intro.html for tips). Include your code in a **new code cell(s) below**.